In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt

In [2]:
#We need to import the empty sectors list here
empty_sectors_list=[]

In [3]:
#Setup linear models based on sector list names
linear_models = empty_sectors_list

In [ ]:
#Essentially create all column names in right spot
for i in range(0, len(sectorPrices)):
    #Get sector Data, training set only
    sectDat= sectorPrices[i][Training_Set]
    #Create a string of price and PE, this will be used for Lm prediction hence tilde
    form = (list(sectDat)[1] + " ~ " + list(sectDat)[3]).astype(str)
    #Continue to build the string, skipping anything in statement 1
    for j in list(sectDat)[3:len(sectDat)]:
        if (j == "EPSGrowthRate" || j == "EPS.Growth.Rate" || j == "PutCallOpenInterest" || j == "InstitutionOwnership"):
            continue()
        else:
            form = form + " + " + j.astype(str)
    #https://stackoverflow.com/questions/29036179/what-is-the-python-equivalent-to-r-predict-function-for-linear-models
    #Now build the linear model with the string we built, this will probably change when I get code to actually
    #test this because of how python works
    linear_models[[i]] = ols(y = form, x = sectDat)
    
    
#Rename the columns with their sector names
list(linear_models) = sectorNames
    

In [ ]:
pValMatrix = pd.dataframe()

In [5]:
#SetRowNames

In [6]:
#SetColumnNames

In [ ]:
for i in range(1, len(linear_models)):
    sTest <- linear_models[[i]].describe()
    sTest <- sTest.transpose()
    for (metric in rownames(pValMatrix)):
        pValMatrix[metric, i] <- sTest[["coefficients"]][2:len(list(sTest[["coefficients"]])),4][metric]

In [ ]:
plotHeatMap = function(df) {
  tmp <- cbind(rownames(df),df)
  tmp <- melt(tmp)
  colnames(tmp) <- c("Factors", "Sectors", "Value")
  pValGGPlot <<- 'ggplot(tmp, aes(x = Sectors, y = Factors, fill = Value)) + geom_tile() +
    scale_fill_gradient2(low = "white", high = "blue", midpoint = 0, space = "Lab") + 
    labs(title = "Cold Plot", subtitle = "The bluer the area, the worse the P-Value",caption = "Rounded to 4 Decimal Places") +
    geom_text(aes(label = round(Value,digits = 4)))'
  eval(parse(text = pValGGPlot))
}

In [ ]:
# png(filename = "Cold Plot.png",width = 862,height = 550,units = "px")
# plotHeatMap(pValMatrix)
# dev.off()
plotHeatMap(pValMatrix)

In [ ]:
write_csv(pValMatrix,file = "output/P Value Matrix.csv")

In [ ]:
coeffMatrix <- as.data.frame(matrix(nrow = nrow(pValMatrix),ncol = length(sectorNames)))
rownames(coeffMatrix) <- rownames(pValMatrix)
colnames(coeffMatrix) <- sectorNames

In [ ]:
for (i in 1:length(linear_models)) {
  sTest <- summary(linear_models[[i]])
  for(metric in rownames(coeffMatrix)) {
    coeffMatrix[metric, i] <- sTest[["coefficients"]][2:nrow(sTest[["coefficients"]]),1][metric]
  }
}

In [ ]:
plotHeatMap2 = function(df) {
  tmp <- cbind(rownames(df),df)
  tmp <- melt(tmp)
  colnames(tmp) <- c("Factors", "Sectors", "Value")
  coeffHeatGGPlot <<- 'ggplot(tmp, aes(x = Sectors, y = Factors, fill = Value)) + geom_tile() +
    scale_fill_gradient2(low = "blue", high = "red", mid = "white", midpoint = 0, space = "Lab") +
    labs(title = "Coeff Heatmap, Linear",caption = "Rounded to 2 Decimal Places") +
    geom_text(aes(label = round(Value,digits = 2)))'
  eval(parse(text = coeffHeatGGPlot))
}

In [ ]:
plotHeatMap2(coeffMatrix)

In [ ]:
plotRSquared = function(lst) {
  tmp <- sapply(lst, function(x) summary(x)$r.squared)
  tmp <- cbind(names(tmp),as.data.frame(tmp))
  colnames(tmp) <- c("Sectors", "RSquared")
  rSquareGGPlot <<- 'ggplot(tmp, aes(x=Sectors,y=RSquared,ymax=1)) + geom_bar(stat = "identity") + geom_text(aes(label = round(RSquared,digits = 4)),nudge_y=.02) + labs(title = "R-Squareds, Linear Model (New)")'
  eval(parse(text = rSquareGGPlot))
}
plotRSquared(linear_models)

In [ ]:
predictedValuesLinear <- emptySectorList

In [ ]:
predictedValuesLinearMat <- as.data.frame(matrix(nrow = nrow(sectorPrices[[1]][["Testing Set"]]), ncol = length(linear_models)))

In [ ]:
for (i in 1:length(linear_models)) {
  preds <- predict.lm(linear_models[[i]],newdata = sectorPrices[[i]][["Testing Set"]][,-1])
  predictedValuesLinear[[i]] <- preds
  predictedValuesLinearMat[,i] <- preds
}

In [ ]:
names(predictedValuesLinear) <- sectorNames
colnames(predictedValuesLinearMat) <- names(predictedValuesLinear)

In [ ]:
accSetLinear <- NULL
for (i in 1:ncol(predictedValuesLinearMat)) {
  testSet <- predictedValuesLinearMat[,i]
  realSet <- sectorPrices[[i]][["Testing Set"]][,1]
  resultSet <- NULL
  for (j in 1:length(testSet)) {
    if ((testSet[j] >= (realSet[j]*(1-GLOBAL_ACCURACY))) && (testSet[j] <= (realSet[j]*(1+GLOBAL_ACCURACY)))) {
      resultSet[j] <- 1
    } else {
      resultSet[j] <- 0
    }
  }
  accSetLinear <- c(accSetLinear,(sum(resultSet)/length(resultSet)))
  print(paste(sectorNames[i]," Accuracy: ",accSetLinear[i],sep = ''))
}
names(accSetLinear) <- sectorNames

In [ ]:
plotAccLinear <- function() {
  tmp <- accSetLinear
  tmp <- pd.concat(names(tmp),as.data_frame(tmp))
  colnames(tmp) <- c("Sectors", "Accuracy")
  rSquareGGPlot <<- 'ggplot(tmp, aes(x=Sectors,y=Accuracy,ymax=1)) + geom_bar(stat = "identity") + geom_text(aes(label = round(Accuracy,digits = 4)),nudge_y=.02) + labs(title = "Accuracy, Linear Model (New)")'
  eval(parse(text = rSquareGGPlot))
}
plotAccLinear()

In [ ]:
size <- size + 1
if (size == 1) {
  accuracyMatrix[1,] <- accSetLinear
  rownames(accuracyMatrix) <- "Linear Regression"
} else {
  accuracyMatrix <- rbind(accuracyMatrix,accSetLinear)
  rownames(accuracyMatrix)[size] <- "Linear Regression"
}

In [7]:
print("Linear Analysis: CURRENTLY BROKEN")

Linear Analysis: CURRENTLY BROKEN
